In [ ]:
import great_expectations as gx

context = gx.get_context()

In [ ]:
import os

my_connection_string = os.getenv(
    "CONNECTION_STRING", "postgresql://example_user@db/gx_example_db"
)

In [ ]:
# Explicitly create data docs site to use filesystem store with known file location.
# This is done to simplify hosting data docs within the containers, the default is to write to a temp directory.

context.add_data_docs_site(
    site_config={
        "class_name": "SiteBuilder",
        "store_backend": {
            "class_name": "TupleFilesystemStoreBackend",
            "base_directory": "/gx/gx_stores/data_docs",
        },
        "site_index_builder": {"class_name": "DefaultSiteIndexBuilder"},
    },
    site_name="local_site_for_hosting",
)

In [ ]:
datasource_name = "my_datasource"
datasource = context.sources.add_sql(
    name=datasource_name, connection_string=my_connection_string
)

In [ ]:
asset_name = "my_asset"
asset_table_name = "nyc_taxi_data"

In [ ]:
table_asset = datasource.add_table_asset(name=asset_name, table_name=asset_table_name)

In [ ]:
batch_request = table_asset.build_batch_request()

In [ ]:
expectation_suite_name = "my_expectation_suite"
context.add_or_update_expectation_suite(expectation_suite_name=expectation_suite_name)

In [ ]:
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name,
)
validator.head()

In [ ]:
validator.expect_column_values_to_not_be_null("pickup_datetime")
validator.expect_column_values_to_be_between("passenger_count", auto=True)

In [ ]:
validator.save_expectation_suite(discard_failed_expectations=False)

In [ ]:
checkpoint = context.add_or_update_checkpoint(
    name="my_quickstart_checkpoint",
    validator=validator,
)

In [ ]:
checkpoint_result = checkpoint.run()

In [ ]:
checkpoint_result.success